# Generic

In [904]:
import math
import base64
import binascii
import numpy as np
import matplotlib.pyplot as plt

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytearray.fromhex(d)
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    
def draw_bytestring(bs):
    """Represent a byte string as a matrix of color
    """
    tmp = bs + bytes(math.ceil(math.sqrt(len(bs)))**2 - len(bs))
    tmp = [int(b) for b in tmp]
    m = [[tmp[i:i+int(math.sqrt(len(tmp)))] for i in range(0, len(tmp), int(math.sqrt(len(tmp))))]]
    matrix = np.matrix(m[0])
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_aspect('equal')
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.binary)
    plt.colorbar()
    plt.show()

## Helpers

In [864]:
import re
import string

def count_letter_freq(text):
    """Count the frequence of each letter in a string
    Return a dict
    """
    letters = {}
    total = 0
    for l in text:
        letters[l.lower()] = letters.get(l.lower(), 0) + 1
        total += 1
    return {l:s/total for l,s in letters.items()}

def hamming_distance(b1, b2):
    distance = 0
    for char1, char2 in zip(b1,b2):
        distance += bin(char1 ^ char2).count('1')
    return distance
assert hamming_distance(b'this is a test', b'wokka wokka!!!') == 37

def count_upper_lower_signs(text):
    """Count the amount of uppercase letters, lowercase letters and other signs
    """
    counter = {'upper': 0, 'lower': 0, 'signs': 0}
    for c in text:
        if c in string.ascii_lowercase:
            counter['lower'] += 1
        elif c in string.ascii_uppercase:
            counter['upper'] += 1
        else:
            counter['signs'] += 1
    return counter

NON_PRINTABLE_REGEX = "[^\w " + string.printable + "\n]"
def is_encodable(bs):
    try:
        return bs.decode()
    except UnicodeDecodeError:
        return False
def is_text(s):
    s_test = re.sub(NON_PRINTABLE_REGEX, "", s.strip())
    return s_test == s.strip()
def is_english(text):
    """Simple scoring function that counts probabilities of caracters and tries
    to match them with the probabilities of the english language"""
    if isinstance(text, (bytes, bytearray)):
        s = is_encodable(text)
    else:
        s = text
    if not s:
        return 0
    freqs = ['etaoin shrdlcumwfgypbvkjxqz', [12.702,9.056,8.167,7.507,6.966,6.749,6.4,6.327,6.094,5.987,4.253,4.025,2.782,2.758,2.406,2.360,2.228,2.015,1.974,1.929,1.492,0.978,0.772,0.153,0.150,0.095,0.074]]
    nominal_scores = {l:s for l,s in zip(freqs[0], freqs[1])}
    text_freqs = count_letter_freq(s)
    score = 0
    for letter, freq in text_freqs.items():
        if letter.lower() in nominal_scores:
            score += nominal_scores[letter.lower()]
        else:
            score -= 1
    return max(score / len(text), 0)


def pad(text, blocksize=16):
    padsize = blocksize - (len(text) % blocksize)
    if padsize == 0:
        padsize = blocksize
    return text + bytes([padsize]*padsize)
assert pad(b"YELLOW SUBMARINE", blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'

def unpad(text):
    """Validate padding and return unpadded value"""
    assert text[-text[-1]:] == bytes([text[-1]]*text[-1])
    return text[:-text[-1]]
assert unpad(pad(b"this is random text", blocksize=7)) == b"this is random text"

## Crypto 101

In [630]:
import math
import itertools

def xor(d1, d2, format):
    b1,b2 = to_bytes(d1, format), to_bytes(d2, format)
    return bytes_to(bytes(char1 ^ char2 for char1,char2 in zip(b1,b2)), format)

def repeating_xor(s, key, format):
    bs = to_bytes(s, format)
    bkey = bytearray(itertools.islice(itertools.cycle(to_bytes(key, format)), len(bs)))
    return xor(bs, bkey, format)

def guess_repeating_xor_keysize(text, count=3, minsize=2, maxsize=40):
    """Try to guess the size of a repeating XOR key from a given ciphertext.
    Return top <count> guesses.
    """
    values = {}
    for keysize in range(minsize, maxsize+1):
        # We use 4 times the keysize in order to get a better reading
        values[keysize] = hamming_distance(text[:keysize*4], text[keysize*4:2*keysize*4]) / (keysize*4)
    values = [(keysize, value) for keysize, value in values.items()]
    values.sort(key=lambda x:x[1]) # The best guess is the lowest hamming distance
    return [keysize for keysize,_ in values[:count]]
    

def crack_single_char_xor(text):
    """Try to crack a cipher as a single character XOR of english text
    """
    bestguess = {'key': 0, 'confidence': -len(text), 'cleartext': ''}
    for i in range(32, 127):
        try:
            score = is_english(xor(text, bytes([i]*len(text)), 'bytes'))
            if score > bestguess['confidence']:
                bestguess['key'] = bytes([i])
                bestguess['confidence'] = score
                bestguess['cleartext'] = xor(text, bytes([i]*len(text)), 'str')
        except UnicodeDecodeError:
            pass
    return bestguess

def crack_repeating_xor(text, keysize):
    """Try to crack a cipher as a repeating key XOR, with a given keysize
    """
    # Split into blocks of size keysize
    blocks = [text[i:i+keysize] for i in range(0, len(text), keysize)]
    
    # Now, refactor blocks to be all encrypted with the same caracter
    # (every n-th caracter of a block is encrypted with the same n-th caracter of the key)
    transposed_blocks = [bytes(block[n] for block in blocks if len(block) > n) for n in range(keysize)]
    
    # Solve every transposed block for single-caracter xor
    cleartext_transposed_blocks = [crack_single_char_xor(block) for block in transposed_blocks]
    
    # Rebuild and return cleartext
    #key = bytes(block['key'][0] for block in cleartext_transposed_blocks)
    key = b''.join(block['key'] for block in cleartext_transposed_blocks)
    cleartext = repeating_xor(text, key, 'str')
    confidence = sum(block['confidence'] for block in cleartext_transposed_blocks) / len(cleartext_transposed_blocks)
    
    return {'key': key, 'cleartext':cleartext, 'confidence':confidence}

## Crypto 102

In [933]:
from Crypto.Cipher import AES

def decrypt_aes_ecb(cipher, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        return unpad(aes.decrypt(cipher))
    else:
        return aes.decrypt(cipher)
def encrypt_aes_ecb(clear, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        clear = pad(clear)
    return aes.encrypt(clear)
assert decrypt_aes_ecb(encrypt_aes_ecb(b'test', 'YELLOW SUBMARINE'), 'YELLOW SUBMARINE') == b'test'

def encrypt_aes_cbc(clear, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = pad(clear)
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    return cipher
def decrypt_aes_cbc(cipher, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    return unpad(clear)
assert decrypt_aes_cbc(encrypt_aes_cbc(b'test'*200, b'YELLOW SUBMARINE'), b'YELLOW SUBMARINE') == b'test'*200
        
        

def detect_ecb(cipher, blocksize=16):
    """Return a score estimating if a given cipher has been enrypted using ECB mode.
    This score is simply a score of redundancy of n-byte blocks"""
    score = 0
    for blockstart in range(0, len(cipher), blocksize):
        # The redundancy count has to be block aligned, because a
        # given cleartext block will only give the same cipher if it
        # is aligned.
        score += cipher.count(cipher[blockstart:blockstart+blocksize])
    return score / len(cipher) * blocksize # Normalize with the number of blocks

# Cryptopals notebook

## Stage 1

### Exercice 1

In [632]:
def hex_to_base64(h):
    return bytes_to(to_bytes(h, 'hex'), 'base64')

s = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
assert hex_to_base64(s) == 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

### Exercice 2

In [633]:
s1 = '1c0111001f010100061a024b53535009181c'
s2 = '686974207468652062756c6c277320657965'
r = '746865206b696420646f6e277420706c6179'

assert xor(s1, s2, 'hex') == r

### Exercice 3

In [634]:
s = to_bytes('1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736', 'hex')
print(crack_single_char_xor(s))

{'confidence': 2.3081764705882355, 'cleartext': "Cooking MC's like a pound of bacon", 'key': b'X'}


### Exercice 4

In [635]:
buest_guess = {'confidence': 0}
with open('set1_4.txt') as f:
    for idx, line in enumerate(f):
        bline = to_bytes(line.strip(), 'hex')
        guess = crack_single_char_xor(bline)
        if guess['confidence'] > buest_guess['confidence']:
            buest_guess = guess
            print('New best guess : %s on line %s with key %s' % (buest_guess['cleartext'], idx, buest_guess['key']))

lKu*ge_H
nXdLm on line 35 with key b'P'
New best guess : Now that the party is jumping
 on line 170 with key b'5'


### Exercice 5

In [636]:
s = to_bytes("Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal", 'str')
k = to_bytes('ICE', 'str')
r = '0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

assert repeating_xor(s, k, 'hex') == r

### Exercice 6

In [637]:
%matplotlib inline

with open('set1_6.txt') as f:
    s = to_bytes(f.read().replace('\n', '').strip(), 'base64')
    
values = guess_repeating_xor_keysize(s, count=10)
print('Top key sizes are : ', values[:3])

# Solve for the the 3 most probable keysizes
for keysize in values[:1]:
    print(crack_repeating_xor(s, keysize))

Top key sizes are :  [29, 5, 12]
{'confidence': 0.9711152490421459, 'cleartext': "I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSuper

### Exercice 7

In [638]:
with open('set1_7.txt') as f:
    s = to_bytes(f.read().replace('\n', '').strip(), 'base64')
    
decrypt_aes_ecb(s, 'YELLOW SUBMARINE')

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

### Exercice 8

In [639]:
%matplotlib inline

s = []
with open('set1_8.txt') as f:
    for line in f:
        s.append(to_bytes(line.strip(), 'hex'))

guess = {'max': detect_ecb(s), 'line': 0}
for idx, l in enumerate(s[1:]):
    score = detect_ecb(l)
    if score > guess['max']:
        guess['max'] = score
        guess['line'] = idx
        print('New best guess :', guess)

New best guess : {'line': 0, 'max': 1.0}
New best guess : {'line': 131, 'max': 2.2}


## Stage 2

### Exercice 9

In [640]:
assert pad(b'YELLOW SUBMARINE', blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'

### Exercice 10

In [641]:
with open('set2_2.txt') as f:
    s = to_bytes(f.read().replace('\n', '').strip(), 'base64')
    
print(decrypt_aes_cbc(s, 'YELLOW SUBMARINE'))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

### Exercice 11

In [642]:
import os
import random

# Encrypt using a random key and a random encryption mode
def encrypt_oracle(clear):
    key = os.urandom(16)
    prefix = os.urandom(random.randint(5,10))
    suffix = os.urandom(random.randint(5,10))
    name = ''
    if random.random() < 0.5:
        name = 'AES_CBC'
        cipher = encrypt_aes_cbc
    else:
        name = 'AES_ECB'
        cipher = encrypt_aes_ecb
    return cipher(prefix + clear + suffix, key, IV=os.urandom(16)), name

def detect_ecb_cbc(encrypt_oracle):
    """Detect if the given encryption_oracle is using CBC or ECB
    """
    text = b'\x00' * 160
    cipher, name = encrypt_oracle(text)
    if detect_ecb(cipher) > 1:
        return name == 'AES_ECB'
    else:
        return name == 'AES_CBC'
    
score = 0
n = 1000
for i in range(n):
    guess = detect_ecb_cbc(encrypt_oracle)
    if guess:
        score += 1
print('Got it correct %s/%s (%s %%)' % (score, n, score/n*100))

Got it correct 1000/1000 (100.0 %)


### Exercice 12

In [764]:
def encrypt_fixed_oracle(clear):
    key = b'\x82\xd5\xb7)\x15r\xd8\xe1f\xe7\xf9\xf1A8=#'
    prefix = b''
    suffix = to_bytes('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK', 'base64')
    return encrypt_aes_ecb(prefix + clear + suffix, key)

# The objective is now to crack the encrypted suffix, without using the key ofc.

# Detect if it is ECB
assert detect_ecb(encrypt_fixed_oracle(b'A'*100)) > 1

# Detect blocksize
def detect_blocksize():
    for blocksize in range(1, 33):
        for filler in range(blocksize):
            # Use oracle to encrypt <blocksize>*A*2 + <filler>*A
            # When the blocksize is correct and the padding actually completes
            # the final block of the oracle prefix, then we have 2 identical
            # blocks at the end
            cipher = encrypt_fixed_oracle(b'A'*blocksize*2 + b'A'*filler)
            if cipher[:blocksize] == cipher[blocksize:blocksize*2]:
                print('Blocksize : %s (pad : %s)' % (blocksize, filler))
                return blocksize

# It turns out, the blocksize is 16 and the oracle appends
# a block-aligned string to the input before encryption.

# Serve a known block that is one byte short to the oracle.
# The last byte in the block will be the first byte of the cleartext
# that is to be attacked.
# Then, encrypt a known block to brute-force the last byte with
# known prefix. This uncovers the first byte of cleartext. Shift
# and repeat after that.

# Detail : 
# Encrypt A*15 to get a cipher block
# Bruteforce A*15+i then compare with above block to recover first byte.
# Encrypt A*14 to get a cipher block
# Bruteforce A*14+first_recovered_cipher_byte+i then compare and get second byte
# ...

cleartext = b''
reference_blocks = {}

def bruteforce_last_block_byte(oracle, reference, prefix, blocksize=16):
    """Bruteforce the last byte of a given block.
    """
    assert len(reference) == blocksize
    assert len(prefix) == blocksize - 1
    for j in range(256):
        a = prefix + bytes([j])
        b = oracle(bytes(a))[:blocksize]
        if b == reference:
            if j == 1:
                continue
            return bytes([j])
    print(reference)
    print(prefix)
    raise ValueError('Bruteforce failed :(')
        

cleartext = b''
cipher = encrypt_fixed_oracle(b'')
for i in range(138): # the cipher is actually only 138 bytes long. If we solve for 144, the padding messes up the bruteforce
    padding = b'A'*(len(cipher)-1-len(cleartext))
    prefix = padding + cleartext
    reference = encrypt_fixed_oracle(padding)[len(padding)-15+i:len(padding)+1+i]
    cleartext += bruteforce_last_block_byte(encrypt_fixed_oracle, reference, prefix=prefix[-15:])
    print(chr(cleartext[-1]), end='')


Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by


### Exercice 13

There is a first a little setup that will emulate a very basic web app cookie generation process

In [813]:
import random

def k_v_parsing(s):
    values = [s2.split('=') for s2 in s.split('&')]
    return {key:value for key,value in values}
assert k_v_parsing('foo=bar&baz=qux&zap=zazzle') == {'foo': 'bar', 'baz':'qux', 'zap':'zazzle'}

def k_v_encoding(d):
    # Add a small sort to the items to ensure consistency
    return 'email={email}&uid={uid}&role={role}'.format(**d)

def profile_for(email, uid=10):
    cleanmail = email.replace('&', '').replace('=', '')  # remove encoding caracters
    return k_v_encoding({'email': cleanmail, 'uid':str(uid), 'role': 'user'})
assert profile_for('foo@bar.com') == 'email=foo@bar.com&uid=10&role=user'

# The super duper app secret
secret = b'\xdeaQ\xc4\xc4\x85\x8b\xf1\xcb\x01dAx\xf2\xea\x86'

def cookie(profile):
    """Generate an AES encrypted cookie from the encoded profile
    """
    return encrypt_aes_ecb(to_bytes(profile, 'str'), key=secret)

def read_cookie(cipher):
    """Extract profile from cookie
    """
    return k_v_parsing(bytes_to(decrypt_aes_ecb(cipher, key=secret), 'str'))

assert read_cookie(cookie('email=foo@bar.com&uid=10&role=user')) == k_v_parsing(profile_for('foo@bar.com', uid=10))

In [814]:
# ECB Cut & Paste

# For that, we use an email that allows the block to end on '&role='
# then cut the last block of the resulting cookie and replace ic
# with another block. The block will be the second block of the
# cookie generated from the email '{A*10}admin{\x0B*11}@bar.com'
# This email generates a padded block with only 'admin' in it.
block1 = cookie(profile_for('A'*10 + 'admin' + '\x0B'*11 + '@bar.com'))[16:32] # second block is the winner
block2 = cookie(profile_for('A'*7 + '@a.com'))[:-16] # shave off last block from this one
print(read_cookie(block2 + block1))

{'email': 'AAAAAAA@a.com', 'uid': '10', 'role': 'admin'}


### Exercice 14

v2 of the fixed oracle ECB attack

In [861]:
import os

def encrypt_fixed_oracle(clear):
    key = b'\x82\xd5\xb7)\x15r\xd8\xe1f\xe7\xf9\xf1A8=#'
    prefix = b"\xa8\xa4?\xe3&\x0c\xa5\xb7\x1a\x8f\xae\xe7\xd9.e\xef\x95\xeb8>\xfa\xba\x14_\xb4\xad-X.Z\xb0\xef\xc3\x8d\xf3\xa3\xd8\x88'\xc4\xe2\t"
    suffix = to_bytes('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK', 'base64')
    return encrypt_aes_ecb(prefix + clear + suffix, key)


# This is basically a twist on the Exercice 12 oracle.
# We just have to find out how long the prefix is. Then,
# we can complete to nearest block and just cut out the
# first few blocks. After that, see exercice 12 for solution.

# 1/ count the bytes that are in common in these two
#    > gives us the prefix to a precision of 1 block
def find_prefix_length():
    pok1 = encrypt_fixed_oracle(b'A'*32)
    pok = []
    last_j = None
    for i in range(16):
        pok.append(encrypt_fixed_oracle(b'A'*i))
        for j in range(len(pok[-1])):
            if pok1[j] != pok[-1][j]:
                if last_j is not None and j != last_j:
                    # The length of identical bytes has changed. The blocked switched.
                    # This means that the previous identical length is part of the
                    # fixed prefix, and that the A padding just attained the block
                    # limit. This gives us the complete length of the prefix.
                    return last_j + (16-i)
                else:
                    last_j = j
                    break
                    
print('The prefix is of length %s' % find_prefix_length())

def wrapped_encrypt_fixed_oracle_factory(oracle, prefix_length, blocksize=16):
    filler = b'A'*(blocksize-(prefix_length%blocksize))
    shave_bytes = prefix_length + len(filler)
    print('Shaving %s bytes of the start and filling with %s bytes' % (shave_bytes, len(filler)))
    def new_oracle(clear):
         return oracle(filler+clear)[shave_bytes:]
    return new_oracle

new_oracle = wrapped_encrypt_fixed_oracle_factory(encrypt_fixed_oracle, find_prefix_length())

# Now that we have a new oracle that actually behaves like the first one !
# Yay \o/ !
# See Exercice 12 for the rest

cleartext = b''
cipher = new_oracle(b'')
for i in range(138): # the cipher is actually only 138 bytes long. If we solve for 144, the padding messes up the bruteforce
    padding = b'A'*(len(cipher)-1-len(cleartext))
    prefix = padding + cleartext
    reference = new_oracle(padding)[len(padding)-15+i:len(padding)+1+i]
    cleartext += bruteforce_last_block_byte(new_oracle, reference, prefix=prefix[-15:])
    print(chr(cleartext[-1]), end='')

The prefix is of length 42
Shaving 48 bytes of the start and filling with 6 bytes
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by


### Exercice 15

In [869]:
assert unpad(b"ICE ICE BABY\x04\x04\x04\x04") == b"ICE ICE BABY"
try:
    unpad(b"ICE ICE BABY\x05\x05\x05\x05")
    assert False
except AssertionError:
    assert True
try:
    unpad(b"ICE ICE BABY\x01\x02\x03\x04")
    assert False
except AssertionError:
    assert True

### Exercice 16

First, define a few functions that emulate how a (web)app would handle userdata.

In [908]:
def build_userdata(data):
    key = b',\xe8~mv\xca\xce\x1c \xd4\x83\x9a\x12\xd2\xd7"'
    d =  pad(b'comment1=cooking%20MCs;userdata=' + data.replace(b'=', b'').replace(b';', b'') + \
            b';comment2=%20like%20a%20pound%20of%20bacon')
    return encrypt_aes_cbc(d, key=key)

def is_admin(encrypted_data):
    key = b',\xe8~mv\xca\xce\x1c \xd4\x83\x9a\x12\xd2\xd7"'
    data = decrypt_aes_cbc(encrypted_data, key=key)
    print(data)
    return b';admin=true;' in data

assert not is_admin(build_userdata(b'test'))

b'comment1=cooking%20MCs;userdata=test;comment2=%20like%20a%20pound%20of%20bacon\x02\x02'


In [945]:
import array

# Build a cipher that will have at least the length to insert A;admin=true, but also be block aligned.
# It so happens that our prefix is block aligned and user data starts on the 3rd block
cipher = build_userdata(b'AadminAtrueAAAAA')

# Now, let's flip some bits !
# Given the XORs used by the CBC decryption function, we can simply flip the
# ciphertext of the previous block to influence the block we have control over.
cipher = xor(b'\x00'*16 + xor(b'A', b';', 'bytes') + b'\x00'*(22-17) + xor(b'A', b'=', 'bytes') + \
                b'\x00'*(27-23) + xor(b'A', b';', 'bytes') + b'\x00'*(len(cipher)-28),
             cipher,
             'bytes')
assert is_admin(cipher)

b'comment1=cooking \xd6\x00\xf0\x8e5\x8e\x91\xb8\xb1\xacf\r?C-;admin=true;AAAA;comment2=%20like%20a%20pound%20of%20bacon\x06\x06\x06\x06\x06\x06'
